<a href="https://colab.research.google.com/github/nabilah-afrin/recommendation_system_rokomri_books/blob/secondary/notebook/preprocessing_bn_books_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Rokomari Recommendation Dataset
# %cd /content/drive/MyDrive/Dataset/rokomari_books

/content/drive/MyDrive/Rokomari Recommendation Dataset


In [3]:
!ls

category_counts.csv	rokomari_bn_books_author_corrected.csv
category_counts.gsheet	rokomari_books_categories_corrected.csv
df_bn.csv


In [4]:
# !pip install googletrans==4.0.0-rc1
!pip install googletrans==4.0.0rc1 --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00


In [6]:
!pip install langdetect -q -q -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# !pip install -U easynmt

In [7]:
!pip install deep_translator --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.1 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import langdetect as ld
# import googletrans as gt
# import deep_translator as dt
# from deep_translator import GoogleTranslator

# Only Bangla Books

In [17]:
# load the rokomari_bn_books.csv
df_bn = pd.read_csv("rokomari_bn_books_author_corrected.csv")


In [18]:
df_bn.head(5)

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
0,লু স্যুন,পশ্চিমবঙ্গের বই: সমকালীন গল্প,9788194097303,No Ratings,No Reviews,320,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,নির্বাচিত গল্প সংকলন,bn,No,bn,bn
1,আর্কাদি গাইদার,পশ্চিমবঙ্গের বই: শিশু-কিশোর উপন্যাস,9788193860939,No Ratings,No Reviews,180,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,তিমুর ও তার দলবল,bn,No,bn,bn
2,মতি নন্দী,পশ্চিমবঙ্গের বই: সমকালীন উপন্যাস,No ISBN,No Ratings,No Reviews,250,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,নায়কের প্রবেশ ও প্রস্থান,bn,No,bn,bn
3,উৎপলকুমার বসু,পশ্চিমবঙ্গের বই: বাংলা কবিতা,No ISBN,No Ratings,No Reviews,300,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,হাঁস চলার পথ,bn,No,bn,bn
4,দিলরুবা হাসান,বয়স যখন ৪-৮: শিক্ষামূলক,No ISBN,No Ratings,No Reviews,129,No image,বৃত্তকলা একাডেমি,0.0,No summary,অনুভূতির ছোঁয়া,bn,No,bn,bn


# Useful Functions

In [4]:
import re
import time
# from easynmt import EasyNMT
# model = EasyNMT('m2m_100_1.2B')
# print(model.translate('Bank Job Entrance Exam Preparation', target_lang='bn'))

def detect_language(row):
    try:
        lang = ld.detect(str(row))
    except:
        return 'unknown'
    return lang

def keep_only_bangla(row):
    """
    The unicode range between \u0980 - \u09FF defines the Bangla characters
    and digits in the Unicode character set
    """
    return re.sub(r'[^\u0980-\u09FF ]+', '', str(row)).strip()

def translate_author_names(row):
    text = str(row)
    # time.sleep(8)
    translator = gt.Translator()
    translation = translator.translate(text=text, dest='bn')

    return translation.text

def translate_to_bangla(row):
    return GoogleTranslator(source='auto', target='bn').translate(text=str(row))

def remove_unnecessary_characters(row):
    # pattern = r'[ও|():/, ]+'
    text = re.sub(r'[:]', ",", str(row))
    text = re.sub(r'[/]', ", ", text)
    text = re.sub(r'[(|)]', "", text)
    text = re.sub(r'\sএবং\s', ', ', text)
    text = re.sub(r'\sও\s', ', ', text)

    return text.strip()



In [ ]:
# text = "শিশু-কিশোর: বই এবং (উপন্যাস) ও কিতাব/পুস্তিকা"
# text = "এসএসসি ১ম বর্ষ"
# text = "যখন ৪-৮: গল্প"
# text = re.sub(r'[:]', ",", str(text))
# text = re.sub(r'[/]', ", ", text)
# text = re.sub(r'[(|)]', "", text)
# text = re.sub(r'\sএবং\s', ', ', text)
# text = re.sub(r'\sও\s', ', ', text)
# # text = re.sub(r' ', "", text)
# print(text)


# 2. Categories


In [ ]:
df_bn['categories'].nunique()

1974

In [ ]:
df_bn['categories'].value_counts().reset_index().loc[:20]

,categories,count
0,বাংলা কবিতা,8621
1,পশ্চিমবঙ্গের বই,7096
2,সমকালীন উপন্যাস,4339
3,সমকালীন গল্প,2756
4,শিশু-কিশোর গল্প,2124
5,চিরায়ত উপন্যাস,1180
6,ছড়া,937
7,পশ্চিমবঙ্গের বই: প্রবন্ধ,924
8,রোমান্টিক কবিতা,842
9,"নবি-রাসুল, সাহাবা, তাবেই ও অলি-আওলিয়া",839


In [ ]:
df_bn.loc[df_bn['categories'] == "Hadith & Sunnah"]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
785,মাওলানা হামিদা পারভীন,Hadith & Sunnah,9843236270,No Ratings,No Reviews,196,https://ds.rokomari.store/rokomari110/ProductN...,Ahsan Publication,0.0,রাসূলুল্লাহ সাল্লাল্লাহু আলাইহি ওয়াসাল্লাম বল...,দারসে হাদীস ৬ষ্ঠ খন্ড,bn,Yes,bn,id
1814,আল হাফিজুল কাবীর আবু বকর আব্দুর রাজ্জাক ইবনে হ...,Hadith & Sunnah,No ISBN,2,No Review,15000,https://ds.rokomari.store/rokomari110/ProductN...,"Al Maktabatul Islami, Boirut",5.0,No summary,আল মুসান্নাফ লি আব্দির রাযযাক আস সান'আনি (১-১২...,bn,No,bn,id
1953,ইমাম মুহিউদ্দীন ইয়াহইয়া আননববী রহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,175,https://ds.rokomari.store/rokomari110/ProductN...,Alif Publications,0.0,No summary,রিয়াদুস সালেহীন (৩য় খণ্ড) শব্দার্থসহ,bn,No,bn,id
1955,ইমাম মুহিউদ্দীন ইয়াহইয়া আননববী রহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,175,https://ds.rokomari.store/rokomari110/ProductN...,Alif Publications,0.0,No summary,রিয়াদুস সালেহীন (৪র্থ খণ্ড) শব্দার্থসহ,bn,No,bn,id
1956,ইমাম মুহিউদ্দীন ইয়াহইয়া আননববী রহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,630,https://ds.rokomari.store/rokomari110/ProductN...,Alif Publications,0.0,No summary,"রিয়াদুস সালেহীন (১ম, ২য়, ৩য় ও ৪র্থ খণ্ড একত্রে...",bn,No,bn,id
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91739,হযরত মাওলানা উসমান গনী রহ,Hadith & Sunnah,No ISBN,2,1 Review,7599,https://ds.rokomari.store/rokomari110/ProductN...,Al-Kousar prokashani,5.0,No summary,নসরুল বারী শরহে সহীহ্‌ বুখারী (১ম খন্ড থেকে ১৩...,bn,No,bn,id
91875,মুফতী এনামুল হক রায়পুরী,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,1320,No image,Al-Kitabah Prokashini,0.0,No summary,"কামূসুল কুতুবিস সিত্তাহ ৫ম খন্ড, ভলিয়ম- ১ম এবং...",bn,No,bn,id
93506,আল্লামা ওলীউদ্দীন আবু আবদুল্লাহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,504,No image,Adhunik Prokasoni,0.0,মিশকাতুল মাসাবীহ হাদীস শাস্ত্র জগতে এক অনবদ্য ...,মিশকাতুল মাসাবীহ ৮ম খন্ড,bn,Yes,bn,id
93511,আল্লামা ওলীউদ্দীন আবু আবদুল্লাহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,448,No image,Adhunik Prokasoni,0.0,মিশকাতুল মাসাবীহ হাদীস শাস্ত্র জগতে এক অনবদ্য ...,মিশকাতুল মাসাবীহ ৭ম খন্ড,bn,Yes,bn,id


## 2.1 Language variations in Categories

In [ ]:
# df_bn['language_categories'] = df_bn['categories'].apply(detect_language)

In [20]:
df_bn['language_categories'].value_counts()

,count
language_categories,
bn,93948
en,2097
id,625
so,359
de,226
af,183
nl,137
fr,119
tl,80


In [ ]:
df_bn.loc[df_bn['language_categories'] == 'ar']

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
8420,শায়খ আব্দুল জাব্বার জাহাঙ্গীর,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,9789848063019,No Ratings,No Reviews,688,https://ds.rokomari.store/rokomari110/ProductN...,দারুল হিকমাহ পাবলিকেশন্স লিমিটেড,0.0,যাকাত সম্পদশালীদের জন্য একটি ফরয ইবাদাত। এটি ই...,যাকাত ডায়েরি,bn,Yes,bn,ar
16986,ড শায়খ আলী তানতাবী রাহ,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No,1 Review,1000,No image,মাকতাবাতুল আফনান,0.0,আলী তান্তাভি রচিত আরবি ভাষায় বিভিন্ন প্রবন্ধ য...,আল আদাবুল আরবী ইনদা আলী তানতাভী ১-৩ খণ্ড,bn,Yes,bn,ar
35227,থানভী ভারত,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,1296,No image,থানভী ভারত,0.0,No summary,তারবিয়াতুস সালিক ভলি. ৪ - বিষয়- ধর্মীয় (কিতাব ...,bn,No,bn,ar
35265,থানভী ভারত,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,378,No image,থানভী ভারত,0.0,No summary,ইসলাহী নেসাব ভলি. ১ - বিষয়- ধর্মীয় (কিতাব কোড-...,bn,No,bn,ar
35267,থানভী ভারত,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,1458,No image,থানভী ভারত,0.0,No summary,ইহয়াউল উলূম (আরবি) ভলি. ৪ - বিষয়- দর্শন (কিতাব...,bn,No,bn,ar
35301,থানভী ভারত,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,972,No image,থানভী ভারত,0.0,No summary,হজ্জাতুল্লাহিল বালিগাহ ভলি. ১ - বিষয়- দর্শন (ক...,bn,No,bn,ar
35303,থানভী ভারত,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,378,No image,থানভী ভারত,0.0,No summary,মাখযানে আখলাক ভলি. ১ - বিষয়- ধর্মীয় (কিতাব কোড...,bn,No,bn,ar
35304,থানভী ভারত,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,810,No image,থানভী ভারত,0.0,No summary,হুসনুল আজীজ ভলি. ৩ - বিষয়- ধর্মীয় (কিতাব কোড-R...,bn,No,bn,ar
63690,মাদানিয়া ভারত,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,1728,No image,মাদানিয়া ভারত,0.0,No summary,তাজাল্লিয়াতে সফদর ভলি. ৬ - বিষয়- ধর্মীয় (কিতাব...,bn,No,bn,ar
68423,জননী বুক ডিপো সম্পাদক,كتب اللغة العربية والأردية (আরবি ও উর্দূ ভাষার...,No ISBN,No Ratings,No Reviews,1786,No image,জননী বুক ডিপো,0.0,No summary,বুখারী শরীফ (১ - ১০ম খন্ড),bn,No,bn,ar


In [ ]:
df_bn.loc[df_bn['language_categories'] == 'en']

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
27,মোঃ আতিকুর রহমান,Bank Job Entrance Exam Preparation,No ISBN,3,No Review,725,https://ds.rokomari.store/rokomari110/ProductN...,A M Publishers,2.33,No summary,ব্যাংকিং ডিপ্লোমা ডাইজেস্ট - পার্ট ১,bn,No,bn,en
48,ডঃ একেএমএ কাদের,"Diary, Letters and Memories",9789843542939,2,1 Review,731,https://ds.rokomari.store/rokomari110/ProductN...,A-Z Foundation,5.00,"The letters, clippings, scribbles, and documen...",ফিরে দেখা ১৯৭১ এর চিঠি ও কথা,bn,Yes,bn,en
314,এ এন এম নূরুল হক,Story: Children and Teens,9789842010637,2,No Review,86,https://ds.rokomari.store/rokomari110/ProductN...,Adorn Books For Children (ABC),5.00,No summary,ঈশান ও তার সুপারহিরো বন্ধুরা,bn,No,bn,en
392,জোহরা বেগম,When 0-4: Primary Learning,9789842010774,1,No Review,172,https://ds.rokomari.store/rokomari110/ProductN...,Adorn Books For Children (ABC),5.00,None of the books of this series is meant for ...,পিকচার উইথ নাম্বার ড্রয়িং ইন প্লেজার,bn,Yes,bn,en
474,অ্যাডভান্সড পাবলিকেশন্স,Class-5 (PEC),No ISBN,No Ratings,No Reviews,599,https://ds.rokomari.store/rokomari110/ProductN...,Advanced Publications,0.00,Written for the students of class 5 of Kinderg...,অ্যাডভান্সড লারনার'স বেসিক ইংলিশ উইথ স্যলুশন -...,bn,Yes,bn,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97910,জনপ্রশাসন মন্ত্রণালয় সম্পাদক,Others book on law,No ISBN,No Ratings,No Reviews,600,https://ds.rokomari.store/rokomari110/ProductN...,Government Publication,0.00,১৯৯৫ সনে এস্টাব্লিশমেন্ট ম্যানুয়েল হালনাগাদ কর...,এস্টাব্লিশমেন্ট ম্যানুয়েল-ভলিয়ম ২,bn,Yes,bn,en
97912,গভার্ণমেন্ট পাবলিকেশন,Others book on law,No ISBN,No Ratings,No Reviews,604,https://ds.rokomari.store/rokomari110/ProductN...,Government Publication,0.00,"সংস্থাপন মন্ত্রণালয় চাকরির নিয়োগনীতি, পদ্ধতি, ...",এস্টাব্লিশমেন্ট ম্যানুয়েল-ভলিউম ১,bn,Yes,bn,en
98005,এয়ার ভাইস মার্শাল এজি মাহমুদ অব,Workplace Memories and Experience,9789849658115,No Ratings,No Reviews,344,https://ds.rokomari.store/rokomari110/ProductN...,Golpokar,0.00,"W.H. Auden বলতেন, প্রত্যেক আত্মজীবনীতে দুটো চর...",মাই ডেসটিনি,bn,Yes,bn,en
98102,মোহাম্মদ তাজাম্মল হুসাইন,Garments & Textile Engineering,9789849111245,4,1 Review,435,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থনীড় প্রকাশনী,3.50,"টেক্সটাইল কলেজ, টেক্সটাইল ইন্সটিটিউট, টেক্সটাই...",গার্মেন্টস কোয়ালিটি ম্যানেজমেন্ট,bn,Yes,bn,en


## 2.2 Fixing categories detected as other language to Bangla and mixed lanugage

### 2.2.1 Fixing categories detected as other language to Bangla

In [21]:
for language in df_bn['language_categories'].value_counts().index:
    if language == 'bn' or language == 'ar':
        continue
    else:
        print(f"Correcting Language: {language}")
        filt = (df_bn['language_categories'] == language, 'categories')
        df_bn.loc[filt] = df_bn.loc[filt].apply(translate_to_bangla)

print("\nDone")

Correcting Language: en
Correcting Language: id


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-c8c19357e2c7>", line 7, in <cell line: 1>
    df_bn.loc[filt] = df_bn.loc[filt].apply(translate_to_bangla)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/series.py", line 4764, in apply
    ).apply()
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py", line 1209, in apply
    return self.apply_standard()
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/apply.py", line 1289, in apply_standard
    mapped = obj._map_values(
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/base.py", line 921, in _map_values
    return algorithms.map_array(arr, mapper, na_action=na_action, convert=convert)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/algorithms.py", line 1814, in map_array
    return lib.map_infer(values, m

TypeError: object of type 'NoneType' has no len()

### 2.2.2 Fixing categories with mixed languages with Bangla

In [ ]:
filt = (df_bn['language_categories'] == 'ar', 'categories')

In [ ]:
df_bn.loc[filt] = df_bn.loc[filt].apply(keep_only_bangla)

In [ ]:
df_bn.loc[filt]

,categories
8420,আরবি ও উর্দূ ভাষার বই
16986,আরবি ও উর্দূ ভাষার বই
35227,আরবি ও উর্দূ ভাষার বই
35265,আরবি ও উর্দূ ভাষার বই
35267,আরবি ও উর্দূ ভাষার বই
35301,আরবি ও উর্দূ ভাষার বই
35303,আরবি ও উর্দূ ভাষার বই
35304,আরবি ও উর্দূ ভাষার বই
63690,আরবি ও উর্দূ ভাষার বই
68423,আরবি ও উর্দূ ভাষার বই


In [ ]:
# save the modified dataframe

# df_bn.to_csv("rokomari_books_categories_corrected.csv", index=False)

# 3 Load the categories corrected dataframe from here

In [10]:
df_bn = pd.read_csv("rokomari_books_categories_corrected.csv")

### 3.1 Use the Language detection model

In [ ]:
# df_bn['language_categories'] = df_bn['categories'].apply(detect_language)

In [ ]:
df_bn['language_categories'].value_counts()

,count
language_categories,
bn,98254


In [ ]:
df_bn.loc[df_bn['language_categories'] == 'de', 'categories'] = "নবী-রাসুল, সাহাবী, তাবেঈ, ওলি-আউলিয়া"

In [ ]:
df_bn.loc[df_bn['language_categories'] == 'fr', 'categories'] = "যখন ৪-৮, উপকথা"

In [ ]:
df_bn.loc[df_bn['language_categories'] == 'vi', 'categories'] = "টোফেল"

In [ ]:
df_bn.loc[df_bn['language_categories'] == 'en', 'categories'] = "উক্তি, বাণী, প্রবাদ"

In [ ]:
# save the modified dataframe

# df_bn.to_csv("rokomari_books_categories_corrected.csv", index=False)

In [ ]:
# df_bn.loc[df_bn['language_categories'] == 'en']

In [ ]:
# category_counts_df = pd.read_csv("/content/drive/MyDrive/Rokomari Recommendation Dataset/category_counts.csv")

In [ ]:
# def filter_cat(text):
#     return ':' in text

# category_counts_df = df_bn[df_bn['categories'].apply(filter_cat)]['categories'].value_counts().reset_index()

# another way to do this
category_counts_df = df_bn.loc[df_bn['categories'].str.contains(":"), 'categories'].value_counts().reset_index()

In [ ]:
# this is not necessary, also it's a good practice not to
# capitalize names

# category_counts_df.columns = ['Category', 'Count']
category_counts_df

,categories,count
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515
...,...,...
800,নবম ও দশম (বিজ্ঞান): ধর্ম ও নৈতিক শিক্ষা পাঠ্য...,1
801,ইঞ্জিনিয়ারিং: পরীক্ষার প্রস্তুতি,1
802,মুক্তিযুদ্ধের ইতিহাস: বিবিধ,1
803,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1


In [ ]:
# category_counts_df.to_csv('category_counts.csv', index=False)

In [ ]:
# df_bn['language_categories'] = df_bn['categories'].apply(detect_language)

In [ ]:
# df_bn['language_categories'].value_counts()

In [ ]:
df_bn.loc[df_bn['language_categories'].isin(["no", "sv", "pt", "vi", "cy"])]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
1010,কবি মমতাজ রেনু,Poem Criticism,9843226488,No Ratings,No Reviews,430,https://ds.rokomari.store/rokomari110/ProductN...,Ahsania Books,0.00,"Ilhami(Inspired) Poetess Mumtaz Renu, a profou...",এঞ্জেল অন মাই শোল্ডারস্,bn,Yes,bn,pt
1012,কবি মমতাজ রেনু,Poem Criticism,No ISBN,No Ratings,No Reviews,1720,https://ds.rokomari.store/rokomari110/ProductN...,Ahsania Books,0.00,No summary,মেরি সুরাত তেরা আয়না,bn,No,bn,pt
9953,এসেনশিয়াল পাবলিকেশন সম্পাদক,TOEFL,No ISBN,7,7 Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,4.86,No summary,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,No,bn,vi
9954,এসেনশিয়াল পাবলিকেশন সম্পাদক,TOEFL,No ISBN,No Ratings,No Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,0.00,CONTENTS,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,Yes,bn,vi
11416,সোনিয়া পারভিন,Cyber Law,No ISBN,No Ratings,No Reviews,174,No image,Hasan Law Books,0.00,No summary,"সাইবার নিরাপত্তা আইন, ২০২৩",bn,No,bn,cy
14331,জয়কলি সম্পাদনা পরিষদ সম্পাদক,TOEFL,No ISBN,No Ratings,No Reviews,213,No image,জয়কলি পাবলিকেশন্স লিঃ,0.00,"সকল প্রশ্নের নির্ভুল উত্তর, সঠিক ব্যাখ্যা, প্র...",বারণ’স এন্ড ক্লিফস টোয়েফল - বাংলা অনুবাদ,bn,Yes,bn,vi
24467,মেজর শের মোহাম্মদ খান অবঃ অনুবাদক,Rhymes,978-984-97262-7-2,No Ratings,No Reviews,129,https://ds.rokomari.store/rokomari110/ProductN...,Raowa Publication,0.00,ছোটো সোনামনিদের ছড়ার বই। এই বইয়ের প্রতিটি ছড়া ...,ছােটােদর বাংলা ছড়া ও English Rhymes,bn,Yes,bn,cy
33028,বিচারপতি মোঃ আজিজুল হক,Cyber Law,9789849156499,No Ratings,No Reviews,898,https://ds.rokomari.store/rokomari110/ProductN...,Universal Book House,0.00,No summary,সাইবার ল এন্ড ক্রাইম,bn,No,bn,cy
33138,অ্যাডভোকেট জাহাঙ্গীর আলম সরকার,Cyber Law,978984892811,No Ratings,No Reviews,627,No image,Unique Law Book House,0.00,"১. সাইবার নিরাপত্তা আইন,২০২৩",সাইবার নিরাপত্তা আইনের ভাষ্য ও পর্নোগ্রাফি নিয়...,bn,Yes,bn,cy
34443,প্রফেসর মোঃ হাফিজুর রহমান,Honors,No ISBN,No Ratings,No Reviews,264,No image,Titas Publications,0.00,No summary,লিনিয়ার এলজাবরা - অনার্স ৩য় বর্ষ,bn,No,bn,no


In [ ]:
df_bn.loc[df_bn['language_categories'].isin(["no", "sv", "pt", "vi", "cy"])].shape

(25, 15)

In [ ]:
category_counts_df = category_counts_df.join(
    category_counts_df['categories'].str.split(':', expand=True).rename(
        columns={0: 'cat_1', 1: 'cat_2', 2: 'cat_3'}
    )
)

In [ ]:
category_counts_df['categories_split'] = category_counts_df['categories'].str.split(":")

In [ ]:
category_counts_df

,categories,count,cat_1,cat_2,cat_3,categories_split
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,None,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]"
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,None,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]"
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,None,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]"
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",None,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ..."
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,None,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]"
...,...,...,...,...,...,...
800,নবম ও দশম (বিজ্ঞান): ধর্ম ও নৈতিক শিক্ষা পাঠ্য...,1,নবম ও দশম (বিজ্ঞান),ধর্ম ও নৈতিক শিক্ষা পাঠ্য সহায়িকা,None,"[নবম ও দশম (বিজ্ঞান), ধর্ম ও নৈতিক শিক্ষা পাঠ..."
801,ইঞ্জিনিয়ারিং: পরীক্ষার প্রস্তুতি,1,ইঞ্জিনিয়ারিং,পরীক্ষার প্রস্তুতি,None,"[ইঞ্জিনিয়ারিং, পরীক্ষার প্রস্তুতি]"
802,মুক্তিযুদ্ধের ইতিহাস: বিবিধ,1,মুক্তিযুদ্ধের ইতিহাস,বিবিধ,None,"[মুক্তিযুদ্ধের ইতিহাস, বিবিধ]"
803,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1,নবম ও দশম (ব্যবসায় শিক্ষা),বাংলাদেশ ও বিশ্ব পরিচয় পাঠ্য সহায়িকা,None,"[নবম ও দশম (ব্যবসায় শিক্ষা), বাংলাদেশ ও বিশ্ব..."


In [ ]:
category_counts_df['category_length'] = category_counts_df['categories_split'].apply(lambda x: len(x))

In [ ]:
category_counts_df.head()

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,None,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]",2
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,None,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]",2
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,None,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",None,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,None,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]",2


In [ ]:
category_counts_df['category_length'].value_counts()

,count
category_length,
2,802
3,3


In [ ]:
# to get the values where cat_3 == None

category_counts_df.loc[(category_counts_df['cat_3'].isna())]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,None,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]",2
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,None,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]",2
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,None,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",None,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,None,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]",2
...,...,...,...,...,...,...,...
800,নবম ও দশম (বিজ্ঞান): ধর্ম ও নৈতিক শিক্ষা পাঠ্য...,1,নবম ও দশম (বিজ্ঞান),ধর্ম ও নৈতিক শিক্ষা পাঠ্য সহায়িকা,None,"[নবম ও দশম (বিজ্ঞান), ধর্ম ও নৈতিক শিক্ষা পাঠ...",2
801,ইঞ্জিনিয়ারিং: পরীক্ষার প্রস্তুতি,1,ইঞ্জিনিয়ারিং,পরীক্ষার প্রস্তুতি,None,"[ইঞ্জিনিয়ারিং, পরীক্ষার প্রস্তুতি]",2
802,মুক্তিযুদ্ধের ইতিহাস: বিবিধ,1,মুক্তিযুদ্ধের ইতিহাস,বিবিধ,None,"[মুক্তিযুদ্ধের ইতিহাস, বিবিধ]",2
803,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1,নবম ও দশম (ব্যবসায় শিক্ষা),বাংলাদেশ ও বিশ্ব পরিচয় পাঠ্য সহায়িকা,None,"[নবম ও দশম (ব্যবসায় শিক্ষা), বাংলাদেশ ও বিশ্ব...",2


`Fill the `None` values with empty ("") string`

In [ ]:
category_counts_df.loc[:, 'cat_3'].fillna("", inplace=True)

In [ ]:
category_counts_df.head()

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]",2
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]",2
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]",2


In [ ]:
category_counts_df.tail()

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
800,নবম ও দশম (বিজ্ঞান): ধর্ম ও নৈতিক শিক্ষা পাঠ্য...,1,নবম ও দশম (বিজ্ঞান),ধর্ম ও নৈতিক শিক্ষা পাঠ্য সহায়িকা,,"[নবম ও দশম (বিজ্ঞান), ধর্ম ও নৈতিক শিক্ষা পাঠ...",2
801,ইঞ্জিনিয়ারিং: পরীক্ষার প্রস্তুতি,1,ইঞ্জিনিয়ারিং,পরীক্ষার প্রস্তুতি,,"[ইঞ্জিনিয়ারিং, পরীক্ষার প্রস্তুতি]",2
802,মুক্তিযুদ্ধের ইতিহাস: বিবিধ,1,মুক্তিযুদ্ধের ইতিহাস,বিবিধ,,"[মুক্তিযুদ্ধের ইতিহাস, বিবিধ]",2
803,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1,নবম ও দশম (ব্যবসায় শিক্ষা),বাংলাদেশ ও বিশ্ব পরিচয় পাঠ্য সহায়িকা,,"[নবম ও দশম (ব্যবসায় শিক্ষা), বাংলাদেশ ও বিশ্ব...",2
804,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ: অনার্স ৪র্থ বর্ষ,1,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ,অনার্স ৪র্থ বর্ষ,,"[ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ, অনার্স ৪র্থ...",2


In [ ]:
category_counts_df.loc[category_counts_df['cat_1'] == "শিশু-কিশোর"]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
23,শিশু-কিশোর: বিবিধ,212,শিশু-কিশোর,বিবিধ,,"[শিশু-কিশোর, বিবিধ]",2
32,শিশু-কিশোর: বিখ্যাত ব্যক্তি ও জীবনী,148,শিশু-কিশোর,বিখ্যাত ব্যক্তি ও জীবনী,,"[শিশু-কিশোর, বিখ্যাত ব্যক্তি ও জীবনী]",2
45,"শিশু-কিশোর: রূপকথা, উপকথা ও লোককাহিনী",107,শিশু-কিশোর,"রূপকথা, উপকথা ও লোককাহিনী",,"[শিশু-কিশোর, রূপকথা, উপকথা ও লোককাহিনী]",2
50,শিশু-কিশোর: রেফারেন্স ও আত্মউন্নয়নমূলক বই,95,শিশু-কিশোর,রেফারেন্স ও আত্মউন্নয়নমূলক বই,,"[শিশু-কিশোর, রেফারেন্স ও আত্মউন্নয়নমূলক বই]",2
67,শিশু-কিশোর: রচনাসমগ্র/সংকলন,70,শিশু-কিশোর,রচনাসমগ্র/সংকলন,,"[শিশু-কিশোর, রচনাসমগ্র/সংকলন]",2
68,শিশু-কিশোর: ছবি আঁকা ও গান,69,শিশু-কিশোর,ছবি আঁকা ও গান,,"[শিশু-কিশোর, ছবি আঁকা ও গান]",2
69,শিশু-কিশোর: সাধারণ জ্ঞান,67,শিশু-কিশোর,সাধারণ জ্ঞান,,"[শিশু-কিশোর, সাধারণ জ্ঞান]",2
80,"শিশু-কিশোর: গণিত, বিজ্ঞান ও প্রযুক্তি",57,শিশু-কিশোর,"গণিত, বিজ্ঞান ও প্রযুক্তি",,"[শিশু-কিশোর, গণিত, বিজ্ঞান ও প্রযুক্তি]",2
85,শিশু-কিশোর: ভাষা আন্দোলন ও মুক্তিযুদ্ধ,54,শিশু-কিশোর,ভাষা আন্দোলন ও মুক্তিযুদ্ধ,,"[শিশু-কিশোর, ভাষা আন্দোলন ও মুক্তিযুদ্ধ]",2


In [ ]:
category_counts_df.loc[category_counts_df['cat_1'] == "অনুবাদ"]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
498,"অনুবাদ: জীবনী, স্মৃতিচারণ ও সাক্ষাৎকার",3,অনুবাদ,"জীবনী, স্মৃতিচারণ ও সাক্ষাৎকার",,"[অনুবাদ, জীবনী, স্মৃতিচারণ ও সাক্ষাৎকার]",2
567,অনুবাদ: প্রবন্ধ,2,অনুবাদ,প্রবন্ধ,,"[অনুবাদ, প্রবন্ধ]",2
593,অনুবাদ: শিশু-কিশোর,2,অনুবাদ,শিশু-কিশোর,,"[অনুবাদ, শিশু-কিশোর]",2
629,অনুবাদ: কমিকস ও নকশা,1,অনুবাদ,কমিকস ও নকশা,,"[অনুবাদ, কমিকস ও নকশা]",2
646,অনুবাদ: ইসলাম,1,অনুবাদ,ইসলাম,,"[অনুবাদ, ইসলাম]",2
657,অনুবাদ: বিবিধ ধর্মীয় বই,1,অনুবাদ,বিবিধ ধর্মীয় বই,,"[অনুবাদ, বিবিধ ধর্মীয় বই]",2
660,অনুবাদ: পৌরাণিক কাহিনী,1,অনুবাদ,পৌরাণিক কাহিনী,,"[অনুবাদ, পৌরাণিক কাহিনী]",2
686,"অনুবাদ: ব্যবসা, বিনিয়োগ ও অর্থনীতি",1,অনুবাদ,"ব্যবসা, বিনিয়োগ ও অর্থনীতি",,"[অনুবাদ, ব্যবসা, বিনিয়োগ ও অর্থনীতি]",2
687,"অনুবাদ:রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও...",1,অনুবাদ,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",,"[অনুবাদ, রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,...",2


In [ ]:
category_counts_df.loc[category_counts_df['cat_2'].str.contains("পাঠ্য সহায়িকা", regex=True)]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
100,এইচএসসি ১ম বর্ষ: গণিত পাঠ্য সহায়িকা,43,এইচএসসি ১ম বর্ষ,গণিত পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, গণিত পাঠ্য সহায়িকা]",2
111,এইচএসসি ১ম বর্ষ: পদার্থবিজ্ঞান পাঠ্য সহায়িকা,38,এইচএসসি ১ম বর্ষ,পদার্থবিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, পদার্থবিজ্ঞান পাঠ্য সহায়িকা]",2
123,এইচএসসি ১ম বর্ষ: রসায়ন পাঠ্য সহায়িকা,33,এইচএসসি ১ম বর্ষ,রসায়ন পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, রসায়ন পাঠ্য সহায়িকা]",2
180,এইচএসসি ১ম বর্ষ: জীববিজ্ঞান পাঠ্য সহায়িকা,21,এইচএসসি ১ম বর্ষ,জীববিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, জীববিজ্ঞান পাঠ্য সহায়িকা]",2
230,এইচএসসি ২য় বর্ষ: পদার্থবিজ্ঞান পাঠ্য সহায়িকা,16,এইচএসসি ২য় বর্ষ,পদার্থবিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, পদার্থবিজ্ঞান পাঠ্য সহায়িকা]",2
284,এইচএসসি ২য় বর্ষ: রসায়ন পাঠ্য সহায়িকা,11,এইচএসসি ২য় বর্ষ,রসায়ন পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, রসায়ন পাঠ্য সহায়িকা]",2
313,এইচএসসি ২য় বর্ষ: গণিত পাঠ্য সহায়িকা,9,এইচএসসি ২য় বর্ষ,গণিত পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, গণিত পাঠ্য সহায়িকা]",2
381,এইচএসসি ২য় বর্ষ: জীববিজ্ঞান পাঠ্য সহায়িকা,6,এইচএসসি ২য় বর্ষ,জীববিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, জীববিজ্ঞান পাঠ্য সহায়িকা]",2
389,এইচএসসি ১ম বর্ষ: তথ্য ও যোগাযোগ প্রযুক্তি পাঠ্...,5,এইচএসসি ১ম বর্ষ,তথ্য ও যোগাযোগ প্রযুক্তি পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, তথ্য ও যোগাযোগ প্রযুক্তি পা...",2
455,এইচএসসি ২য় বর্ষ: সমাজবিজ্ঞান/সমাজকর্ম পাঠ্য সহ...,4,এইচএসসি ২য় বর্ষ,সমাজবিজ্ঞান/সমাজকর্ম পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, সমাজবিজ্ঞান/সমাজকর্ম পাঠ্য ...",2


In [ ]:
category_counts_df.loc[category_counts_df['cat_2'].str.contains("অনার্স", regex=True)]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
87,রাষ্ট্রবিজ্ঞান বিভাগ: অনার্স ১ম বর্ষ,52,রাষ্ট্রবিজ্ঞান বিভাগ,অনার্স ১ম বর্ষ,,"[রাষ্ট্রবিজ্ঞান বিভাগ, অনার্স ১ম বর্ষ]",2
104,ভূগোল বিভাগ: অনার্স ১ম বর্ষ,41,ভূগোল বিভাগ,অনার্স ১ম বর্ষ,,"[ভূগোল বিভাগ, অনার্স ১ম বর্ষ]",2
110,রাষ্ট্রবিজ্ঞান বিভাগ: অনার্স ৪র্থ বর্ষ,38,রাষ্ট্রবিজ্ঞান বিভাগ,অনার্স ৪র্থ বর্ষ,,"[রাষ্ট্রবিজ্ঞান বিভাগ, অনার্স ৪র্থ বর্ষ]",2
117,ইতিহাস বিভাগ: অনার্স ১ম বর্ষ,36,ইতিহাস বিভাগ,অনার্স ১ম বর্ষ,,"[ইতিহাস বিভাগ, অনার্স ১ম বর্ষ]",2
118,রাষ্ট্রবিজ্ঞান বিভাগ: অনার্স ২য় বর্ষ,35,রাষ্ট্রবিজ্ঞান বিভাগ,অনার্স ২য় বর্ষ,,"[রাষ্ট্রবিজ্ঞান বিভাগ, অনার্স ২য় বর্ষ]",2
...,...,...,...,...,...,...,...
753,অর্থনীতি বিভাগ: অনার্স ৩য় বর্ষ,1,অর্থনীতি বিভাগ,অনার্স ৩য় বর্ষ,,"[অর্থনীতি বিভাগ, অনার্স ৩য় বর্ষ]",2
755,পরিসংখ্যান বিভাগ: অনার্স ২য় বর্ষ,1,পরিসংখ্যান বিভাগ,অনার্স ২য় বর্ষ,,"[পরিসংখ্যান বিভাগ, অনার্স ২য় বর্ষ]",2
767,লোক প্রশাসন বিভাগ: অনার্স ৩য় বর্ষ,1,লোক প্রশাসন বিভাগ,অনার্স ৩য় বর্ষ,,"[লোক প্রশাসন বিভাগ, অনার্স ৩য় বর্ষ]",2
772,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ: অনার্স ৩য় বর্ষ,1,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ,অনার্স ৩য় বর্ষ,,"[ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ, অনার্স ৩য় ব...",2


In [ ]:
category_counts_df.loc[category_counts_df['cat_1'].str.contains("নবম ও দশম", regex=True)]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
121,নবম ও দশম (এসএসসি): কমন সাবজেক্ট,34,নবম ও দশম (এসএসসি),কমন সাবজেক্ট,,"[নবম ও দশম (এসএসসি), কমন সাবজেক্ট]",2
134,নবম ও দশম (এসএসসি): আবশ্যিক বিষয়,28,নবম ও দশম (এসএসসি),আবশ্যিক বিষয়,,"[নবম ও দশম (এসএসসি), আবশ্যিক বিষয়]",2
142,নবম ও দশম (বিজ্ঞান): পদার্থবিজ্ঞান পাঠ্য সহায়িকা,26,নবম ও দশম (বিজ্ঞান),পদার্থবিজ্ঞান পাঠ্য সহায়িকা,,"[নবম ও দশম (বিজ্ঞান), পদার্থবিজ্ঞান পাঠ্য সহা...",2
168,নবম ও দশম (বিজ্ঞান): রসায়ন পাঠ্য সহায়িকা,23,নবম ও দশম (বিজ্ঞান),রসায়ন পাঠ্য সহায়িকা,,"[নবম ও দশম (বিজ্ঞান), রসায়ন পাঠ্য সহায়িকা]",2
205,নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ,19,নবম ও দশম (এসএসসি),বিজ্ঞান বিভাগ,,"[নবম ও দশম (এসএসসি), বিজ্ঞান বিভাগ]",2
241,নবম ও দশম (এসএসসি): ব্যবসায় শিক্ষা বিভাগ,14,নবম ও দশম (এসএসসি),ব্যবসায় শিক্ষা বিভাগ,,"[নবম ও দশম (এসএসসি), ব্যবসায় শিক্ষা বিভাগ]",2
256,নবম ও দশম (এসএসসি): বিজ্ঞান বিভাগ পাঠ্য সহায়িকা,13,নবম ও দশম (এসএসসি),বিজ্ঞান বিভাগ পাঠ্য সহায়িকা,,"[নবম ও দশম (এসএসসি), বিজ্ঞান বিভাগ পাঠ্য সহায...",2
275,নবম ও দশম (এসএসসি): মানবিক বিভাগ পাঠ্য সহায়িকা,11,নবম ও দশম (এসএসসি),মানবিক বিভাগ পাঠ্য সহায়িকা,,"[নবম ও দশম (এসএসসি), মানবিক বিভাগ পাঠ্য সহায়...",2
276,নবম ও দশম (বিজ্ঞান): জীববিজ্ঞান পাঠ্য সহায়িকা,11,নবম ও দশম (বিজ্ঞান),জীববিজ্ঞান পাঠ্য সহায়িকা,,"[নবম ও দশম (বিজ্ঞান), জীববিজ্ঞান পাঠ্য সহায়িকা]",2
329,নবম ও দশম (এসএসসি): ব্যবসায় শিক্ষা বিভাগ পাঠ্...,8,নবম ও দশম (এসএসসি),ব্যবসায় শিক্ষা বিভাগ পাঠ্য সহায়িকা,,"[নবম ও দশম (এসএসসি), ব্যবসায় শিক্ষা বিভাগ পা...",2


### 3.2 Stripping unnecessary characters

In [11]:
df_bn.loc[:, 'categories_fixed'] = df_bn['categories'].apply(remove_unnecessary_characters)

In [12]:
df_bn.loc[:, 'categories_fixed']

,categories_fixed
0,"পশ্চিমবঙ্গের বই, সমকালীন গল্প"
1,"পশ্চিমবঙ্গের বই, শিশু-কিশোর উপন্যাস"
2,"পশ্চিমবঙ্গের বই, সমকালীন উপন্যাস"
3,"পশ্চিমবঙ্গের বই, বাংলা কবিতা"
4,"বয়স যখন ৪-৮, শিক্ষামূলক"
...,...
98249,"বয়স যখন ৪-৮, বাংলা ছড়া"
98250,"বয়স যখন ৮-১২, গল্প"
98251,"পশ্চিমবঙ্গের বই, রচনাসমগ্র, সংকলন"
98252,হিন্দু ধর্মীয় বই


In [13]:
df_bn.loc[:, 'categories_fixed'][0].split(", ")

['পশ্চিমবঙ্গের বই', 'সমকালীন গল্প']

In [ ]:
category_counts_df['cat_1'].value_counts()

,count
cat_1,
পশ্চিমবঙ্গের বই,98
এইচএসসি ১ম বর্ষ,42
এইচএসসি ২য় বর্ষ,37
শিশু-কিশোর,32
বয়স যখন ৮-১২,28
...,...
"ভর্তি, নিয়োগ ও প্রস্তুতি পরীক্ষা",1
বয়স যখন ৪-৮,1
ইতিহাসে বাংলাদেশ,1


In [14]:
df_bn.loc[df_bn['categories_fixed'].str.contains("-", regex=True)]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed
1,আর্কাদি গাইদার,পশ্চিমবঙ্গের বই: শিশু-কিশোর উপন্যাস,9788193860939,No Ratings,No Reviews,180,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,তিমুর ও তার দলবল,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, শিশু-কিশোর উপন্যাস"
4,দিলরুবা হাসান,বয়স যখন ৪-৮: শিক্ষামূলক,No ISBN,No Ratings,No Reviews,129,No image,বৃত্তকলা একাডেমি,0.0,No summary,অনুভূতির ছোঁয়া,bn,No,bn,bn,"বয়স যখন ৪-৮, শিক্ষামূলক"
8,কল্যাণী রায়,পশ্চিমবঙ্গের বই: শিশু-কিশোর বই,9788195063413,No Ratings,No Reviews,324,No image,বুক রিডার্স,0.0,No summary,ছবি ছড়া কত মজা,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, শিশু-কিশোর বই"
75,সম্বিতা দত্ত,পশ্চিমবঙ্গের বই: শিশু-কিশোর বই,No ISBN,2,No Review,101,https://ds.rokomari.store/rokomari110/ProductN...,অভিযান পাবলিশার্স (ভারত),5.0,কিছু অংশ,ফুরবার্গের ফন্দিবাজ,bn,Yes,bn,bn,"পশ্চিমবঙ্গের বই, শিশু-কিশোর বই"
124,শুভজিৎ গুপ্ত,শিশু-কিশোর গল্প,No ISBN,1,No Review,160,No image,অভিযান পাবলিশার্স (ভারত),5.0,No summary,ভুজাং যখন ভূত,bn,No,bn,bn,শিশু-কিশোর গল্প
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98229,মওলানা আজাদ সুবহানি,"নবি-রাসুল, সাহাবা, তাবেই ও অলি-আওলিয়া",9789849884101,No Ratings,No Reviews,430,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থিক প্রকাশন,0.0,মওলানা আজাদ সুবহানির সাথে পরিচিত হই মওলানা ভাস...,বিপ্লবী নবী,bn,Yes,bn,bn,"নবি-রাসুল, সাহাবা, তাবেই, অলি-আওলিয়া"
98231,মোস্তাক আহ্মাদ,"শিশু-কিশোর: গণিত, বিজ্ঞান ও প্রযুক্তি",9789849295020,No Ratings,No Reviews,170,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থ কুটির,0.0,ছােটদের জ্যামিতি শিক্ষার সহজ পাঠের চিন্তা থেক...,ছোটদের জ্যামিতি শিক্ষা,bn,Yes,bn,bn,"শিশু-কিশোর, গণিত, বিজ্ঞান, প্রযুক্তি"
98244,সুমন মাহবুব,শিশু-কিশোর গল্প,9789848273654,1,No Review,78,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থ প্রকাশ,5.0,No summary,অদ্ভুত সব গল্প,bn,No,bn,bn,শিশু-কিশোর গল্প
98249,আ শ ম বাবর আলী,বয়স যখন ৪-৮: বাংলা ছড়া,9789849338253,No Ratings,No Reviews,129,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থ প্রকাশ,0.0,No summary,আবোল তাবোল ছড়া,bn,No,bn,bn,"বয়স যখন ৪-৮, বাংলা ছড়া"


In [ ]:
category_1 = category_counts_df['cat_1'].value_counts()

In [ ]:
category_1[5:41]

,count
cat_1,
বয়স যখন ১২-১৭,26
When 4-8,14
বয়স যখন ৪-৮,13
অনুবাদ,12
মুক্তিযুদ্ধের ইতিহাস,11
বয়স যখন ৪-৮,11
When 8-12,10
ব্যবস্থাপনা বিভাগ,10
উদ্ভিদবিজ্ঞান বিভাগ,9


In [ ]:
category_counts_df.loc[category_counts_df['cat_1'] == "Foreign Language Books"]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
362,Foreign Language Books: Other Foreign Books,6,Foreign Language Books,Other Foreign Books,None,"[Foreign Language Books, Other Foreign Books]",2
558,Foreign Language Books: Islamic Books,2,Foreign Language Books,Islamic Books,None,"[Foreign Language Books, Islamic Books]",2
627,Foreign Language Books: Religious Books,1,Foreign Language Books,Religious Books,None,"[Foreign Language Books, Religious Books]",2
680,"Foreign Language Books: Mathematics, Science &...",1,Foreign Language Books,"Mathematics, Science & Technology",None,"[Foreign Language Books, Mathematics, Science...",2
683,Foreign Language Books: Self-help and meditation,1,Foreign Language Books,Self-help and meditation,None,"[Foreign Language Books, Self-help and medita...",2
719,"Foreign Language Books: Professional, Journal ...",1,Foreign Language Books,"Professional, Journal and Reference",None,"[Foreign Language Books, Professional, Journa...",2


In [ ]:
df_bn.loc[df_bn['categories'].str.contains(r'^Foreign Language Books:', regex=True)]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
20496,মুনসি প্রেমচাঁদ,Foreign Language Books: Other Foreign Books,9798187335282,1,No Review,360,No image,Nai Sadi Book House,3.0,No summary,গোডান (হিন্দি),bn,No,bn,en
29632,এ আর কে শর্মা,Foreign Language Books: Other Foreign Books,9789383606115,1,No Review,180,https://ds.rokomari.store/rokomari110/ProductN...,Sri Sarada Book House (India),5.0,No summary,স্ট্রেন্থ ফার্স্ট গুডনেস নেক্সট,bn,No,bn,en
31080,পাওলো কোয়েলহো,Foreign Language Books: Other Foreign Books,9788183280921,No Ratings,No Reviews,405,No image,Wisdom Tree (India),0.0,No summary,জাহির (হিন্দি),bn,No,bn,en
33418,ড ভি আবদুর রহিম,"Foreign Language Books: Professional, Journal ...",No ISBN,2,No Review,7600,No image,U.K. Islamic Academy,3.0,No summary,এরাবিক কোর্স ফর ইংলিশ স্পিকিং স্টুডেন্টস (৩ভলি...,bn,No,bn,en
62024,বি কে শিবানী,Foreign Language Books: Other Foreign Books,9788182748576,1,No Review,351,https://ds.rokomari.store/rokomari110/ProductN...,Manjul Publishing House Pvt. Ltd (India),2.0,No summary,আসেম আনন্দ কি আর (হ্যাপিনেস আনলিমিটেড),bn,No,bn,en
62028,রোন্ডা বাইর্ন,Foreign Language Books: Other Foreign Books,9788183220941,1,No Review,718,https://ds.rokomari.store/rokomari110/ProductN...,Manjul Publishing House Pvt. Ltd (India),5.0,No summary,দ্যা সিক্রেট)(হিন্দি),bn,No,bn,en
66124,খান স্যার পাটনা,Foreign Language Books: Other Foreign Books,No ISBN,2,No Review,225,https://ds.rokomari.store/rokomari110/ProductN...,Kiran Prakashan,5.0,No summary,এনসিইআরটি ৩০০০ ফ্যাক্টস,bn,No,bn,en
76148,শায়খ সফিউর রহমান মোবারকপুরী,Foreign Language Books: Islamic Books,9786035000710,No Ratings,No Reviews,3960,No image,Darussalam (India),0.0,No summary,কালেকশন ফ্রম রিয়াদ উস সালেহীন,bn,No,bn,en
76156,দেভ আল কারদায়ী,Foreign Language Books: Religious Books,No ISBN,No Ratings,No Reviews,2800,No image,Darussalam (India),0.0,No summary,ডিকশনারি অব ইসলামিক টার্ম,bn,No,bn,en
94196,প্রসন্ত কুমার গিরি,"Foreign Language Books: Mathematics, Science &...",9789387162051,No Ratings,No Reviews,765,No image,Academic Publishers (Indian),0.0,No summary,ইন্ট্রোডাকশন টু স্ট্যাটিসটিক্স ইনক্লুডিং স্ট্য...,bn,No,bn,en


In [ ]:
print(category_1[category_1 == 1])

cat_1
কম্পিউটার সায়েন্স অ্যান্ড টেকনোলজি                      1
এইচএসসি মানবিক বিভাগ                                    1
Poem and Rhyme                                          1
Others                                                  1
Children & Teen Magazine                                1
বইমেলা ২০১৩                                             1
Self-Development                                        1
এনভায়রনমেন্টাল ইঞ্জিনিয়ারিং                             1
রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভেঞ্চার    1
Islamic Book                                            1
রকমারি নির্বাচিত বই                                     1
বিদেশী ফটোগ্রাফি ম্যাগাজিন                              1
History                                                 1
Self-Help, Motivational and Meditation                  1
HSC and Equivalent                                      1
টেস্ট পেপারস                                            1
Social and Political Magazine                           1
HSC Sugg

In [ ]:
category_counts_df['cat_2'].value_counts()

,count
cat_2,
অনার্স ১ম বর্ষ,24
অনার্স ৩য় বর্ষ,22
অনার্স ২য় বর্ষ,22
অনার্স ৪র্থ বর্ষ,20
ডিগ্রি ১ম বর্ষ,19
...,...
ইন্টারভিউ,1
জীববিজ্ঞান পাঠ্য সহায়িকা,1
মানবিক বিভাগ পাঠ্য সহায়িকা,1


In [ ]:
category_2 = category_counts_df['cat_2'].value_counts()
print(category_2[category_2 == 1])

cat_2
 Science Department Pattho Sohayika             1
 রান্না, স্বাস্থ্য, ফ্যাশন ও পরিবার বিষয়ক বই    1
আইসিটি পাঠ্য সহায়িকা                           1
 হিসাববিজ্ঞান পাঠ্য সহায়িকা                    1
 এসএসসি ও সমমান                                 1
                                               ..
 ইন্টারভিউ                                      1
 জীববিজ্ঞান পাঠ্য সহায়িকা                      1
 মানবিক বিভাগ পাঠ্য সহায়িকা                    1
 বিবিধ প্রসঙ্গ                                  1
 ইংরেজি পাঠ্য সহায়িকা                          1
Name: count, Length: 262, dtype: int64


In [25]:
df_bn['category_list'] = df_bn['category_list'].apply(lambda x: x.split(', '))


In [28]:
df_bn['category_list'].apply(len).max()

8

In [29]:
df_bn['list_length'] = df_bn['category_list'].apply(len)

In [33]:
df_bn.loc[df_bn['list_length'] == 8]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,category_list,list_length
22,জেমস হেডলি চেজ,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",No ISBN,No Ratings,No Reviews,180,https://ds.rokomari.store/rokomari110/ProductN...,এ পি পি (ভারত),0.00,No summary,স্বপ্নের খেলাঘর,bn,No,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
2042,গিলিয়ান ফ্লিন,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",9789849367017,12,10 Reviews,413,https://ds.rokomari.store/rokomari110/ProductN...,অন্যধারা,3.58,কে আমরা? কি-বা করতে পেরেছি এতদিন আমরা একে অপর...,গন গার্ল,bn,Yes,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
2079,ড্যান ব্রাউন,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",9848331506,8,1 Review,336,https://ds.rokomari.store/rokomari110/ProductN...,অন্যধারা,4.88,‘দ্য ডা ভিঞ্চি কোড' প্রকাশিত হওয়ার পর ড্যান ...,দ্য লস্ট সিম্বল,bn,Yes,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
2156,ড্যান ব্রাউন,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",9789845033152,16,5 Reviews,413,https://ds.rokomari.store/rokomari110/ProductN...,অন্যধারা,3.75,No summary,অরিজিন,bn,No,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
2495,স্টিফেন কিং,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",9789849582939,No Ratings,No Reviews,150,https://ds.rokomari.store/rokomari110/ProductN...,অন্যধারা,0.00,মাঝবয়সি স্যান্ডারসন তার পিতার সঙ্গেদেখা করতে হ...,কিংস - ২য় খণ্ড,bn,Yes,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97595,অনীশ দাস অপু অনুবাদক,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",9847035501895,1,1 Review,274,https://ds.rokomari.store/rokomari110/ProductN...,জিনিয়াস পাবলিকেশন্স,5.00,অনেকগুলাে হরর ও ভৌতিক গল্প দিয়ে সাজানাে হয়ে...,টেরর ফ্রম দ্য ডীপস্‌ অ্যান্ড আদার হরর স্টোরিজ,bn,Yes,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
97602,অনীশ দাস অপু,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",9789848059647,2,1 Review,196,https://ds.rokomari.store/rokomari110/ProductN...,জিনিয়াস পাবলিকেশন্স,3.50,No summary,পৃথিবী সেরা থ্রিলার গল্প,bn,No,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
97663,পাভেল ক্লুশান্ৎসেভ,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",9789849139898,No Ratings,No Reviews,188,https://ds.rokomari.store/rokomari110/ProductN...,গোল্ডেন বুকস,0.00,No summary,মহাশূন্যের রূপকথা,bn,No,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
97780,অরুণ কুমার বিশ্বাস অনুবাদক,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",984150099X,No Ratings,No Reviews,72,https://ds.rokomari.store/rokomari110/ProductN...,গ্লোব লাইব্রেরী (প্রাঃ) লি.,0.00,No summary,ক্যামডেন কিলার,bn,No,bn,bn,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভে...","[রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,, অ্যাডভ...",8
